In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)

In [52]:
current_date = pd.Timestamp.today()  # Example: 2024-04-15

In [60]:
periods_current = _create_period_columns(pd.Series([current_date]))
print(periods_current["week"].astype(int))

0    2879
dtype: int64


In [30]:
def _create_period_columns(date_series, calendar_type="standard"):
    config = {
        "standard": {
            "year": "Y",
            "quarter": "Q",
            "month": "M",
            "week": "W",
        },
        "fiscal": {
            "fiscal_year": "Y-FEB",
            "fiscal_quarter": "Q-FEB",
        },
    }

    periods = {}
    for key, freq in config[calendar_type].items():
        periods[key] = date_series.dt.to_period(freq)

    return periods

In [31]:
def _create_period_boundaries(periods):
    boundaries = {}

    for key, period in periods.items():
        boundaries.update(
            {
                f"start_of_{key}": period.dt.start_time,
                f"end_of_{key}": period.dt.end_time.dt.normalize(),
            }
        )

    return boundaries

In [48]:
def _calculate_offset(periods, current_period):
    return periods.dt.ordinal - current_period.ordinal

In [51]:
def _add_standard_columns(df):
    dt = df["date"].dt
    periods = _create_period_columns(df["date"], "standard")

    return df.assign(
        **_create_period_boundaries(periods),
        year_offset=periods["year"].ordinal,
    )


def _add_fiscal_columns(df):
    dt = df["date"].dt
    periods = _create_period_columns(dt, "fiscal")

    return df.assign(
        **_create_period_boundaries(periods),
    )


# def _add_standard_columns(df):
#     dt = df["date"].dt
#     iso = dt.isocalendar()
#     period_w = dt.to_period("W")
#     period_m = dt.to_period("M")
#     period_q = dt.to_period("Q")
#     period_y = dt.to_period("Y")

#     current_date = pd.Timestamp.today()
#     current_period_w = current_date.to_period("W")
#     current_period_m = current_date.to_period("M")
#     current_period_q = current_date.to_period("Q")
#     current_period_y = current_date.to_period("Y")

#     return df.assign(
#         year=dt.year,
#         month=dt.month,
#         month_abbr=dt.strftime("%b"),
#         year_month=dt.strftime("%b %Y"),
#         year_month_number=dt.year * 12 + dt.month - 1,
#         quarter=dt.quarter,
#         year_quarter=dt.to_period("Q"),
#         iso_week=iso.week,
#         day_of_week=iso.day,
#         day_name=dt.strftime("%a"),
#         start_of_year=period_y.dt.start_time,
#         end_of_year=period_y.dt.end_time.dt.floor("D"),
#         start_of_quarter=period_q.dt.start_time,
#         end_of_quarter=period_q.dt.end_time.dt.floor("D"),
#         start_of_month=period_m.dt.start_time,
#         end_of_month=period_m.dt.end_time.dt.floor("D"),
#         start_of_week=period_w.dt.start_time,
#         end_of_week=period_w.dt.end_time.dt.floor("D"),
#         year_offset=dt.year - current_period_y.year,
#         quarter_offset=(
#             ((dt.year - current_date.year) * 4)
#             + (dt.quarter - current_date.quarter)
#         ),
#         month_offset=(
#             ((dt.year - current_date.year) * 12)
#             + (dt.month - current_date.month)
#         ),
#         week_offset=(
#             period_w.dt.start_time - current_period_w.start_time
#         ).dt.days
#         // 7,
#     )


# def _add_fiscal_columns(df):
#     dt = df["date"].dt

#     period_y = dt.to_period("Y-FEB")
#     period_q = dt.to_period("Q-FEB")
#     fiscal_start_month = period_y.dt.start_time.dt.month

#     current_date = pd.Timestamp.today()
#     current_period_y = current_date.to_period("Y-FEB")
#     current_period_q = current_date.to_period("Q-FEB")

#     return df.assign(
#         fiscal_year=period_y.dt.year,
#         fiscal_month=((dt.month - fiscal_start_month) % 12) + 1,
#         fiscal_quarter=period_q.dt.quarter,
#         fiscal_year_quarter=period_q,
#         start_of_fiscal_year=period_y.dt.start_time,
#         end_of_fiscal_year=period_y.dt.end_time.dt.normalize(),
#         start_of_fiscal_quarter=period_q.dt.start_time,
#         end_of_fiscal_quarter=period_q.dt.end_time.dt.floor("D"),
#         fiscal_year_offset=period_y.dt.year - current_period_y.year,
#         fiscal_quarter_offset=(
#             ((period_y.dt.year - current_period_y.year) * 4)
#             + (period_q.dt.quarter - current_period_q.quarter)
#         ),
#     )

df = pd.DataFrame(
    {
        "date": pd.date_range(
            start="2024-12-01",
            end="2025-03-31",
            freq="D",
        )
    }
)
df = df.pipe(_add_standard_columns)
display(df)

AttributeError: 'Series' object has no attribute 'ordinal'